In [1]:
from sklearn.cluster import KMeans
import numpy as np
import matplotlib.pyplot as plt

from matplotlib import cm
import matplotlib.cbook as cbook
import matplotlib.colors as colors

import pandas as pd
import xarray as xr
from netCDF4 import Dataset, num2date, date2num
from datetime import datetime, timedelta
import os

import seaborn as sns
import cmocean as cmocn

%matplotlib inline

In [3]:
ds= xr.open_dataset('/srv/data2/srai_poseidon/srai_poseidon/observation/SatelliteVsBuoy/downloads/Buoy/extractedGZ2/WINDS/T_09N_140W_xrr_MatchUp.nc')
ds

<xarray.Dataset>
Dimensions:                    (AS_TIME: 1236, NUMCELLS: 3, HEIGHT: 3,
                                TAO_TIME_INDEX: 25, DEPTH: 1)
Coordinates:
  * AS_TIME                    (AS_TIME) datetime64[ns] 2007-01-02T06:57:30 ....
  * NUMCELLS                   (NUMCELLS) int64 0 1 2
  * HEIGHT                     (HEIGHT) float32 3.0 4.0 10.0
  * TAO_TIME_INDEX             (TAO_TIME_INDEX) int64 -12 -11 -10 ... 10 11 12
  * DEPTH                      (DEPTH) float32 1.0
Data variables: (12/139)
    wvc_index                  (AS_TIME, NUMCELLS) float64 ...
    model_speed                (AS_TIME, NUMCELLS) float64 ...
    model_dir                  (AS_TIME, NUMCELLS) float64 ...
    ice_prob                   (AS_TIME, NUMCELLS) float64 ...
    ice_age                    (AS_TIME, NUMCELLS) float64 ...
    wvc_quality_flag           (AS_TIME, NUMCELLS) float64 ...
    ...                         ...
    mean Ux_10N 120min         (AS_TIME) float64 ...
    std. dev. Ux_10N 120min    (AS_TIME) float64 ...
    mean Vy_10N 120min         (AS_TIME) float64 ...
    std. dev. Vy_10N 120min    (AS_TIME) float64 ...
    mean WDIR 120min           (AS_TIME) float64 ...
    std. dev. WDIR 120min      (AS_TIME) float64 ...

In [4]:
list(ds.keys())

['wvc_index',
 'model_speed',
 'model_dir',
 'ice_prob',
 'ice_age',
 'wvc_quality_flag',
 'wind_speed',
 'wind_dir',
 'bs_distance',
 'UWND',
 'VWND',
 'WDIR',
 'WDIR_DM',
 'WDIR_QC',
 'WSPD',
 'WSPD_DM',
 'WSPD_QC',
 'SST',
 'SST_DM',
 'SST_QC',
 'AIRT',
 'AIRT_DM',
 'AIRT_QC',
 'RELH',
 'RELH_DM',
 'RELH_QC',
 'WSPD_10N',
 'WSPD_10',
 'TAO_TIME',
 'mean WSPD 20min',
 'std. dev. WSPD 20min',
 'mean WSPD_10N 20min',
 'std. dev. WSPD_10N 20min',
 'mean Ux_10N 20min',
 'std. dev. Ux_10N 20min',
 'mean Vy_10N 20min',
 'std. dev. Vy_10N 20min',
 'mean WDIR 20min',
 'std. dev. WDIR 20min',
 'mean WSPD 30min',
 'std. dev. WSPD 30min',
 'mean WSPD_10N 30min',
 'std. dev. WSPD_10N 30min',
 'mean Ux_10N 30min',
 'std. dev. Ux_10N 30min',
 'mean Vy_10N 30min',
 'std. dev. Vy_10N 30min',
 'mean WDIR 30min',
 'std. dev. WDIR 30min',
 'mean WSPD 40min',
 'std. dev. WSPD 40min',
 'mean WSPD_10N 40min',
 'std. dev. WSPD_10N 40min',
 'mean Ux_10N 40min',
 'std. dev. Ux_10N 40min',
 'mean Vy_10N 40min

In [17]:
def getSubDF(ds, LAT, LON):
    subds = xr.Dataset()

    taoTimeVar = ['TAO_TIME']
    
    windVars = ['WDIR',
         'WDIR_QC',
         'WDIR_DM',
         'WSPD',
         'WSPD_QC',
         'WSPD_DM',
         'UWND',
         'VWND']

    sstVars = ['SST',
             'SST_QC',
             'SST_DM']
    
    airtVars = ['AIRT',
             'AIRT_QC',
             'AIRT_DM']
    
    rhVars = ['RELH',
             'RELH_QC',
             'RELH_DM']
    
    u10Vars =['WSPD_10N',
             'WSPD_10']

    statVars = ['mean WSPD 20min',
                 'std. dev. WSPD 20min',
                 'mean WSPD_10N 20min',
                 'std. dev. WSPD_10N 20min',
                 'mean Ux_10N 20min',
                 'std. dev. Ux_10N 20min',
                 'mean Vy_10N 20min',
                 'std. dev. Vy_10N 20min',
                 'mean WDIR 20min',
                 'std. dev. WDIR 20min',
                 'mean WSPD 30min',
                 'std. dev. WSPD 30min',
                 'mean WSPD_10N 30min',
                 'std. dev. WSPD_10N 30min',
                 'mean Ux_10N 30min',
                 'std. dev. Ux_10N 30min',
                 'mean Vy_10N 30min',
                 'std. dev. Vy_10N 30min',
                 'mean WDIR 30min',
                 'std. dev. WDIR 30min',
                 'mean WSPD 40min',
                 'std. dev. WSPD 40min',
                 'mean WSPD_10N 40min',
                 'std. dev. WSPD_10N 40min',
                 'mean Ux_10N 40min',
                 'std. dev. Ux_10N 40min',
                 'mean Vy_10N 40min',
                 'std. dev. Vy_10N 40min',
                 'mean WDIR 40min',
                 'std. dev. WDIR 40min',
                 'mean WSPD 50min',
                 'std. dev. WSPD 50min',
                 'mean WSPD_10N 50min',
                 'std. dev. WSPD_10N 50min',
                 'mean Ux_10N 50min',
                 'std. dev. Ux_10N 50min',
                 'mean Vy_10N 50min',
                 'std. dev. Vy_10N 50min',
                 'mean WDIR 50min',
                 'std. dev. WDIR 50min',
                 'mean WSPD 60min',
                 'std. dev. WSPD 60min',
                 'mean WSPD_10N 60min',
                 'std. dev. WSPD_10N 60min',
                 'mean Ux_10N 60min',
                 'std. dev. Ux_10N 60min',
                 'mean Vy_10N 60min',
                 'std. dev. Vy_10N 60min',
                 'mean WDIR 60min',
                 'std. dev. WDIR 60min',
                 'mean WSPD 70min',
                 'std. dev. WSPD 70min',
                 'mean WSPD_10N 70min',
                 'std. dev. WSPD_10N 70min',
                 'mean Ux_10N 70min',
                 'std. dev. Ux_10N 70min',
                 'mean Vy_10N 70min',
                 'std. dev. Vy_10N 70min',
                 'mean WDIR 70min',
                 'std. dev. WDIR 70min',
                 'mean WSPD 80min',
                 'std. dev. WSPD 80min',
                 'mean WSPD_10N 80min',
                 'std. dev. WSPD_10N 80min',
                 'mean Ux_10N 80min',
                 'std. dev. Ux_10N 80min',
                 'mean Vy_10N 80min',
                 'std. dev. Vy_10N 80min',
                 'mean WDIR 80min',
                 'std. dev. WDIR 80min',
                 'mean WSPD 90min',
                 'std. dev. WSPD 90min',
                 'mean WSPD_10N 90min',
                 'std. dev. WSPD_10N 90min',
                 'mean Ux_10N 90min',
                 'std. dev. Ux_10N 90min',
                 'mean Vy_10N 90min',
                 'std. dev. Vy_10N 90min',
                 'mean WDIR 90min',
                 'std. dev. WDIR 90min',
                 'mean WSPD 100min',
                 'std. dev. WSPD 100min',
                 'mean WSPD_10N 100min',
                 'std. dev. WSPD_10N 100min',
                 'mean Ux_10N 100min',
                 'std. dev. Ux_10N 100min',
                 'mean Vy_10N 100min',
                 'std. dev. Vy_10N 100min',
                 'mean WDIR 100min',
                 'std. dev. WDIR 100min',
                 'mean WSPD 110min',
                 'std. dev. WSPD 110min',
                 'mean WSPD_10N 110min',
                 'std. dev. WSPD_10N 110min',
                 'mean Ux_10N 110min',
                 'std. dev. Ux_10N 110min',
                 'mean Vy_10N 110min',
                 'std. dev. Vy_10N 110min',
                 'mean WDIR 110min',
                 'std. dev. WDIR 110min',
                 'mean WSPD 120min',
                 'std. dev. WSPD 120min',
                 'mean WSPD_10N 120min',
                 'std. dev. WSPD_10N 120min',
                 'mean Ux_10N 120min',
                 'std. dev. Ux_10N 120min',
                 'mean Vy_10N 120min',
                 'std. dev. Vy_10N 120min',
                 'mean WDIR 120min',
                 'std. dev. WDIR 120min']
    
    
    satVars = ['wvc_index',
             'model_speed',
             'model_dir',
             'ice_prob',
             'ice_age',
             'wvc_quality_flag',
             'wind_speed',
             'wind_dir',
             'bs_distance']

     
    
    for windVar in windVars:
        subds[windVar] = ds[windVar].sel(HEIGHT=4)
    subds.drop_vars('HEIGHT')
    for sstVar in sstVars:
        subds[sstVar] = ds[sstVar].sel(DEPTH=1)
    subds.drop_vars('DEPTH')
    for rhVar in rhVars:
        subds[rhVar] = ds[rhVar].sel(HEIGHT=3)
    subds.drop_vars('HEIGHT')
    for airtVar in airtVars:
        subds[airtVar] = ds[airtVar].sel(HEIGHT=3)
    subds.drop_vars('HEIGHT')
    for u10Var in u10Vars:
        subds[u10Var] = ds[u10Var].sel(HEIGHT=10)
    subds.drop_vars('HEIGHT')

    for statVar in statVars:
        subds[statVar] = ds[statVar]

    subds['LATITUDE'] = xr.DataArray(LAT * np.ones(len(subds['AS_TIME']), dtype = float), dims=['AS_TIME'])
    subds['LONGITUDE'] = xr.DataArray(LON * np.ones(len(subds['AS_TIME']), dtype = float), dims=['AS_TIME'])
    
    subds['TAO_TIME'] = ds['TAO_TIME']

    deployFileName = f'../../downloads/Buoy/extractedGZ2/WINDS/T_{lat:02d}{latUnits}_{lon:03d}{lonUnits}/T_{lat:02d}{latUnits}_{lon:03d}{lonUnits}_DeploymentDates.nc'
    ds2 = xr.open_dataset(deployFileName)
    startDates = np.array(ds2['startDate'])
    endDates = np.array(ds2['endDate'])

    depNum = np.zeros((len(subds['AS_TIME'])), dtype=int)
    c = 1
    for i in range(1,len(startDates)):
        mask = subds['AS_TIME'] >= startDates[i]
        mask *= subds['AS_TIME'] <= endDates[i]
        depNum[mask] = c
        c = c+1

    subds['Deployment index'] = xr.DataArray(depNum, dims=['AS_TIME'])
    
    
    satDS = ds[satVars]
    satDS = satDS.isel(NUMCELLS=0)

    
    
    returnDS = xr.merge((subds, satDS))

    
    
    return returnDS

In [18]:
latList = [-9, -8, -5, -2, 0, 2, 5, 8, 9]
lonList = [-95, -110, -125, -140, -155, -170, -180, 165]

ylen = len(latList)
xlen = len(lonList)

taskList = []

for latId  in range(ylen):
    for lonId in range(xlen):
        taskList.append([latList[latId], lonList[lonId]])

ntasks = len(taskList)


allDS = xr.Dataset()
dataCount = 0
fileCount = 0
for task in taskList:
    lat = task[0]
    lon = task[1]

    LAT = lat
    LON = lon

    if lat < 0:
        latUnits = 'S'
    else:
        latUnits = 'N'

    if lon < 0:
        lonUnits = 'W'
    else:
        lonUnits = 'E'

    LON = (LON+360)%360
    lat=abs(lat)
    lon=abs(lon)

    matchFname = f'../../downloads/Buoy/extractedGZ2/WINDS/T_{lat:02d}{latUnits}_{lon:03d}{lonUnits}_xrr_MatchUp.nc'
    #deployFileName = f'../../downloads/Buoy/extractedGZ/WINDS/T_{lat:02d}{latUnits}_{lon:03d}{lonUnits}/T_{lat:02d}{latUnits}_{lon:03d}{lonUnits}_DeploymentDates.nc'
    
    if os.path.isfile(matchFname):
        print(f'T_{lat:02d}{latUnits}_{lon:03d}{lonUnits}_xrr_MatchUp_2000.nc')
        rds = xr.open_dataset(matchFname)
        ds = getSubDF(rds, LAT, LON)
        AS_TIME = ds['AS_TIME'].to_numpy()
        ds = ds.rename({'AS_TIME':'count'})
        dataLen = len(ds['count'])
        print(dataCount, dataCount+dataLen)
        ds['count'] = xr.DataArray(pd.Series(np.arange(dataCount, dataCount+dataLen)), 
                                   dims = ['count'])
        ds['AS_TIME'] = xr.DataArray(AS_TIME, 
                                   dims = ['count'])
        dataCount += dataLen
        if fileCount == 0:
            allDS = ds
        else:
            allDS = xr.concat((allDS, ds), dim='count')
        fileCount += 1
        

T_08S_095W_xrr_MatchUp_2000.nc
0 1261
T_08S_110W_xrr_MatchUp_2000.nc
1261 1759


/home/shikhar.rai/anaconda3/envs/mpiNetCDF/lib/python3.11/site-packages/xarray/core/concat.py:546: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))
/home/shikhar.rai/anaconda3/envs/mpiNetCDF/lib/python3.11/site-packages/xarray/core/concat.py:546: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))


T_08S_125W_xrr_MatchUp_2000.nc
1759 3183


/home/shikhar.rai/anaconda3/envs/mpiNetCDF/lib/python3.11/site-packages/xarray/core/concat.py:546: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))
/home/shikhar.rai/anaconda3/envs/mpiNetCDF/lib/python3.11/site-packages/xarray/core/concat.py:546: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))


T_08S_155W_xrr_MatchUp_2000.nc
3183 4722


...

/home/shikhar.rai/anaconda3/envs/mpiNetCDF/lib/python3.11/site-packages/xarray/core/concat.py:546: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))
/home/shikhar.rai/anaconda3/envs/mpiNetCDF/lib/python3.11/site-packages/xarray/core/concat.py:546: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))


T_08N_165E_xrr_MatchUp_2000.nc
44706 45750


/home/shikhar.rai/anaconda3/envs/mpiNetCDF/lib/python3.11/site-packages/xarray/core/concat.py:546: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))
/home/shikhar.rai/anaconda3/envs/mpiNetCDF/lib/python3.11/site-packages/xarray/core/concat.py:546: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))


T_09N_140W_xrr_MatchUp_2000.nc
45750 46986


/home/shikhar.rai/anaconda3/envs/mpiNetCDF/lib/python3.11/site-packages/xarray/core/concat.py:546: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))
/home/shikhar.rai/anaconda3/envs/mpiNetCDF/lib/python3.11/site-packages/xarray/core/concat.py:546: FutureWarning: unique with argument that is not not a Series, Index, ExtensionArray, or np.ndarray is deprecated and will raise in a future version.
  common_dims = tuple(pd.unique([d for v in vars for d in v.dims]))


In [19]:
allDS = allDS.drop_vars(['HEIGHT','DEPTH','NUMCELLS'])

In [20]:
allDS

<xarray.Dataset>
Dimensions:                    (count: 46986, TAO_TIME_INDEX: 25)
Coordinates:
  * count                      (count) int64 0 1 2 3 ... 46982 46983 46984 46985
  * TAO_TIME_INDEX             (TAO_TIME_INDEX) int64 -12 -11 -10 ... 10 11 12
Data variables: (12/143)
    WDIR                       (count, TAO_TIME_INDEX) float32 307.5 ... 237.0
    WDIR_QC                    (count, TAO_TIME_INDEX) float32 2.0 2.0 ... 2.0
    WDIR_DM                    (count, TAO_TIME_INDEX) |S1 b'D' b'D' ... b'D'
    WSPD                       (count, TAO_TIME_INDEX) float32 6.83 7.0 ... 8.5
    WSPD_QC                    (count, TAO_TIME_INDEX) float32 2.0 2.0 ... 2.0
    WSPD_DM                    (count, TAO_TIME_INDEX) |S1 b'D' b'D' ... b'D'
    ...                         ...
    ice_age                    (count) float64 nan nan nan nan ... nan nan nan
    wvc_quality_flag           (count) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    wind_speed                 (count) float64 8.08 6.95 7.07 ... 9.2 10.52
    wind_dir                   (count) float64 312.6 291.8 295.9 ... 235.3 232.5
    bs_distance                (count) float64 -0.1 -0.1 -0.2 ... -0.2 -0.4 -1.3
    AS_TIME                    (count) datetime64[ns] 2007-04-19T04:21:15 ......

In [24]:
np.array(np.unique(allDS['wvc_quality_flag'].to_numpy()), dtype=int)


array([      0,    2048,   65536,  139328,  204864, 1179648, 1181696,
       1187904, 4194304, 4227072])

In [8]:
# extended_flag_meaning = ['rain_correction_not_applied_flag',
# 'correction_produced_negative_spd_flag',
# 'all_ambiguities_contribute_to_nudging_flag',
# 'large_rain_correction_flag',
# 'coastal_processing_applied_flag',
# 'lake_winds_flag',
# 'rain_nearby_flag',
# 'ice_nearby_flag',
# 'significant_rain_correction_flag',
# 'rain_correction_applied_flag',
# 'wind_retrieval_possibly_corrupted_flag']

# extended_flags = np.array(allDS['eflags'].to_numpy(), dtype=int)
# eflags = np.array(['{:016b}'.format(val) for val in extended_flags])
# rainCorrectionNotApplied = np.array([val[15] == '0' for val in eflags])
# rainCorrectionApplied = np.array([val[4] == '1' for val in eflags])
# significantRainCorrectionApplied = np.array([val[5] == '1' for val in eflags])
# largeRainCorrectionApplied = np.array([val[12] == '1' for val in eflags])
# rainMask = np.logical_or(rainCorrectionApplied, significantRainCorrectionApplied)
# rainMask = np.logical_or(rainMask, largeRainCorrectionApplied)
# rainMask = np.logical_or(rainMask, rainCorrectionNotApplied)

In [25]:
# np.sum(largeRainCorrectionApplied), np.sum(significantRainCorrectionApplied), np.sum(rainCorrectionApplied)

In [27]:
centerData = allDS.sel(TAO_TIME_INDEX = 0)
centerData = centerData.drop('TAO_TIME_INDEX')
#indices = list(np.nonzero(~rainMask)[0])

#goodData = centerData.isel(count=~rainMask)
goodData = centerData.isel(count = centerData["WSPD_QC"].isin([1,2]))
goodData = goodData.isel(count = goodData["WDIR_QC"].isin([1,2]))
goodData = goodData.isel(count = goodData["SST_QC"].isin([1,2]))
goodData = goodData.isel(count = goodData["RELH_QC"].isin([1,2]))
goodData = goodData.isel(count = goodData["AIRT_QC"].isin([1,2]))


print(len(goodData['count']))




30222


In [28]:
mask = np.logical_or(goodData['WSPD'] > 1000, goodData['WDIR'] > 3600)
#mask = np.logical_or(mask, abs(goodData['WSPD_10N'].to_numpy()) < 0.1)
mask = np.logical_or(mask, np.isnan(goodData['WSPD'].to_numpy()))
mask = np.logical_or(mask, np.isnan(goodData['WDIR'].to_numpy()))
mask = np.logical_or(mask, np.isnan(goodData['wind_speed'].to_numpy()))
mask = np.logical_or(mask, np.isnan(goodData['wind_dir'].to_numpy()))
print(np.sum(mask))
df = goodData.isel(count = ~mask)
#df = df.to_dataframe()

<xarray.DataArray 'WSPD' ()>
array(19)


In [29]:
df

<xarray.Dataset>
Dimensions:                    (count: 30203)
Coordinates:
  * count                      (count) int64 0 1 2 3 ... 46561 46562 46563 46564
Data variables: (12/143)
    WDIR                       (count) float32 308.1 292.0 293.7 ... 229.0 223.0
    WDIR_QC                    (count) float32 2.0 2.0 2.0 2.0 ... 2.0 2.0 2.0
    WDIR_DM                    (count) |S1 b'D' b'D' b'D' ... b'D' b'D' b'D'
    WSPD                       (count) float32 7.09 6.54 6.59 ... 9.51 9.11 8.63
    WSPD_QC                    (count) float32 2.0 2.0 2.0 2.0 ... 2.0 2.0 2.0
    WSPD_DM                    (count) |S1 b'D' b'D' b'D' ... b'D' b'D' b'D'
    ...                         ...
    ice_age                    (count) float64 nan nan nan nan ... nan nan nan
    wvc_quality_flag           (count) float64 0.0 0.0 0.0 0.0 ... 0.0 0.0 0.0
    wind_speed                 (count) float64 8.08 6.95 7.07 ... 9.41 8.59 9.17
    wind_dir                   (count) float64 312.6 291.8 295.9 ... 233.1 232.4
    bs_distance                (count) float64 -0.1 -0.1 -0.2 ... -0.1 -0.2 -0.1
    AS_TIME                    (count) datetime64[ns] 2007-04-19T04:21:15 ......

In [30]:
df = df.rename_vars({'WSPD':'Wind Speed (TAO)',
                  'WDIR':'Wind Direction (TAO)',
                  'UWND': 'Zonal Wind Speed (TAO)',
                  'VWND': 'Meridional Wind Speed (TAO)',
                  'SST': 'Sea Surface Temperature (TAO)',
                  'RELH': 'Relative Humidity (TAO)',
                  'AIRT': 'Air Temperature (TAO)',
                  'WSPD_10N': 'Neutral Wind Speed at 10m (TAO)',
                  'WSPD_10': 'Wind Speed at 10m (TAO)',
                  'wind_speed': 'Wind Speed (ASCAT)',
                  'wind_dir': 'Wind Direction (ASCAT)'})

In [31]:
df['Speed Difference (ASCAT - TAO)'] = df['Wind Speed (ASCAT)'] - df['Neutral Wind Speed at 10m (TAO)']

## change true degrees to degrees where 0 deg mean east and 90 deg mean north 
df['Wind Direction (TAO)'] = (-(df['Wind Direction (TAO)'] - 90.0) + 360)%360
df['Wind Direction (ASCAT)'] = (-(df['Wind Direction (ASCAT)'] - 90.0) + 360)%360

## calculate zonal and meridional winds 
df['Zonal Neutral Wind Speed (TAO)'] = df['Neutral Wind Speed at 10m (TAO)'] * np.cos(np.deg2rad(df['Wind Direction (TAO)']))
df['Meridional Neutral Wind Speed (TAO)'] = df['Neutral Wind Speed at 10m (TAO)'] * np.sin(np.deg2rad(df['Wind Direction (TAO)']))

df['Zonal Wind Speed (ASCAT)'] = df['Wind Speed (ASCAT)'] * np.cos(np.deg2rad(df['Wind Direction (ASCAT)']))
df['Meridional Wind Speed (ASCAT)'] = df['Wind Speed (ASCAT)'] * np.sin(np.deg2rad(df['Wind Direction (ASCAT)']))

## direction diff in range (-180,180)
dirDiff = ((df['Wind Direction (ASCAT)'] - df['Wind Direction (TAO)'])+360)%360
dirDiff = xr.where(dirDiff > 180, dirDiff-360, dirDiff)
df['Direction Difference (ASCAT - TAO)'] = dirDiff

df['Zonal Wind Speed Difference (ASCAT - TAO)'] = df['Zonal Wind Speed (ASCAT)'] - df['Zonal Neutral Wind Speed (TAO)']
df['Meridional Wind Speed Difference (ASCAT - TAO)'] = df['Meridional Wind Speed (ASCAT)'] - df['Meridional Neutral Wind Speed (TAO)']

for timeWindow in range(20,130,10):
    df[f'Speed Difference (ASCAT - TAO {timeWindow} min mean)'] = df['Wind Speed (ASCAT)'] - df[f'mean WSPD_10N {timeWindow}min']

    ## change true degrees to degrees where 0 deg mean east and 90 deg mean north 
    df[f'mean WDIR {timeWindow}min'] = (-(df[f'mean WDIR {timeWindow}min'] - 90.0) + 360)%360

    ## direction diff in range (-180,180)
    dirDiff = ((df['Wind Direction (ASCAT)'] - df[f'mean WDIR {timeWindow}min'])+360)%360
    dirDiff = xr.where(dirDiff > 180, dirDiff-360, dirDiff)
    df[f'Direction Difference (ASCAT - TAO {timeWindow} min mean)'] = dirDiff

    ### 
    df[f'Zonal Wind Speed Difference (ASCAT - TAO {timeWindow} min mean)'] = df['Zonal Wind Speed (ASCAT)'] - df[f'mean Ux_10N {timeWindow}min']
    df[f'Meridional Wind Speed Difference (ASCAT - TAO {timeWindow} min mean)'] = df['Zonal Wind Speed (ASCAT)'] - df[f'mean Vy_10N {timeWindow}min']

In [32]:
df

<xarray.Dataset>
Dimensions:                                                      (count: 30203)
Coordinates:
  * count                                                        (count) int64 ...
Data variables: (12/195)
    Wind Direction (TAO)                                         (count) float32 ...
    WDIR_QC                                                      (count) float32 ...
    WDIR_DM                                                      (count) |S1 ...
    Wind Speed (TAO)                                             (count) float32 ...
    WSPD_QC                                                      (count) float32 ...
    WSPD_DM                                                      (count) |S1 ...
    ...                                                           ...
    Zonal Wind Speed Difference (ASCAT - TAO 110 min mean)       (count) float64 ...
    Meridional Wind Speed Difference (ASCAT - TAO 110 min mean)  (count) float64 ...
    Speed Difference (ASCAT - TAO 120 min mean)                  (count) float64 ...
    Direction Difference (ASCAT - TAO 120 min mean)              (count) float64 ...
    Zonal Wind Speed Difference (ASCAT - TAO 120 min mean)       (count) float64 ...
    Meridional Wind Speed Difference (ASCAT - TAO 120 min mean)  (count) float64 ...

In [33]:
df.to_netcdf('BuoyDataBadQualityRemovedMatchup_ASCAT.nc')

In [ ]:
len(df['count'])

In [ ]:
selAllDS = allDS.sel(count=df['count'])

In [ ]:
selAllDS = selAllDS.rename_vars({'WSPD':'Wind Speed (TAO)',
                  'WDIR':'Wind Direction (TAO)',
                  'UWND': 'Zonal Wind Speed (TAO)',
                  'VWND': 'Meridional Wind Speed (TAO)',
                  'SST': 'Sea Surface Temperature (TAO)',
                  'RELH': 'Relative Humidity (TAO)',
                  'AIRT': 'Air Temperature (TAO)',
                  'WSPD_10N': 'Neutral Wind Speed at 10m (TAO)',
                  'WSPD_10': 'Wind Speed at 10m (TAO)',
                  'wind_speed': 'Wind Speed (ASCAT)',
                  'wind_dir': 'Wind Direction (ASCAT)'})

In [ ]:
selAllDS['Speed Difference (ASCAT - TAO)'] = selAllDS['Wind Speed (ASCAT)'] - selAllDS['Wind Speed (TAO)']

selAllDS['Wind Direction (TAO)'] = (-(selAllDS['Wind Direction (TAO)'] - 90.0) + 360)%360
selAllDS['Wind Direction (ASCAT)'] = (-(selAllDS['Wind Direction (ASCAT)'] - 90.0) + 360)%360

selAllDS['Direction Difference (ASCAT - TAO 30 min mean)'] = selAllDS['Wind Direction (ASCAT)'] - selAllDS['mean WSPD 30min']
selAllDS['Direction Difference (ASCAT - TAO 30 min mean)'] = selAllDS['Wind Direction (ASCAT)'] - selAllDS['mean WDIR 30min']

selAllDS['Direction Difference (ASCAT - TAO 1 hr mean)'] = selAllDS['Wind Direction (ASCAT)'] - selAllDS['mean WSPD 60min']
selAllDS['Direction Difference (ASCAT - TAO 1 hr mean)'] = selAllDS['Wind Direction (ASCAT)'] - selAllDS['mean WDIR 60min']

selAllDS['Direction Difference (ASCAT - TAO 2 hr mean)'] = selAllDS['Wind Direction (ASCAT)'] - selAllDS['mean WSPD 120min']
selAllDS['Direction Difference (ASCAT - TAO 2 hr mean)'] = selAllDS['Wind Direction (ASCAT)'] - selAllDS['mean WDIR 120min']


selAllDS['Zonal Neutral Wind Speed at 10m (TAO)'] = selAllDS['Neutral Wind Speed at 10m (TAO)']*np.cos(np.deg2rad(selAllDS['Wind Direction (TAO)']))
selAllDS['Meridional Neutral Wind Speed at 10m (TAO)'] = selAllDS['Neutral Wind Speed at 10m (TAO)']*np.sin(np.deg2rad(selAllDS['Wind Direction (TAO)']))

selAllDS['Zonal Neutral Wind Speed at 10m (ASCAT)'] = selAllDS['Wind Speed (ASCAT)']*np.cos(np.deg2rad(selAllDS['Wind Direction (ASCAT)']))
selAllDS['Meridional Neutral Wind Speed at 10m (ASCAT)'] = selAllDS['Wind Speed (ASCAT)']*np.sin(np.deg2rad(selAllDS['Wind Direction (ASCAT)']))

selAllDS['Zonal Wind Speed Difference (ASCAT - TAO)'] = selAllDS['Zonal Neutral Wind Speed at 10m (ASCAT)'] - selAllDS['Zonal Neutral Wind Speed at 10m (TAO)']
selAllDS['Meridional Wind Speed Difference (ASCAT - TAO)'] = selAllDS['Meridional Neutral Wind Speed at 10m (ASCAT)'] - selAllDS['Meridional Neutral Wind Speed at 10m (TAO)']


In [ ]:
selAllDS